# Trabalho 1

In [21]:
"""
Instalação de libs. 
Necessário para primeira execução
"""
# !pip install pyopengl
# !pip install glfw
# !pip install pyglm

'\nInstalação de libs. \nNecessário para primeira execução\n'

In [1]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from numpy import random
from PIL import Image

from shader_s import Shader

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

altura = 700
largura = 700

window = glfw.create_window(largura, altura, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfw.terminate()
    
glfw.make_context_current(window)



In [3]:
ourShader = Shader("vertex_shader.vs", "fragment_shader.fs")
ourShader.use()

program = ourShader.getProgram()

In [4]:
# camera
cameraPos   = glm.vec3(0.0, 0.0, -3.0)
cameraFront = glm.vec3(0.0, 0.0, 0.0) # tanto faz o valor aqui, já que ele vai ser alterado depois a partir da posição do mouse na cena.
cameraUp    = glm.vec3(0.0, 1.0, 0.0)

firstMouse = True
yaw   = -90.0	# yaw is initialized to -90.0 degrees since a yaw of 0.0 results in a direction vector pointing to the right so we initially rotate a bit to the left.
pitch =  0.0
lastX =  largura / 2.0
lastY =  altura / 2.0
fov   =  45.0

# timing
deltaTime = 0.0	# time between current frame and last frame
lastFrame = 0.0


firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2


def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp

    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)
    
    cameraSpeed = 5 * deltaTime
    if key == glfw.KEY_W and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += cameraSpeed * cameraFront
    
    if key == glfw.KEY_S and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= cameraSpeed * cameraFront
    
    if key == glfw.KEY_A and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == glfw.KEY_D and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        

def framebuffer_size_callback(window, largura, altura):

    # make sure the viewport matches the new window dimensions note that width and 
    # height will be significantly larger than specified on retina displays.
    glViewport(0, 0, largura, altura)

# glfw: whenever the mouse moves, this callback is called
# -------------------------------------------------------
def mouse_callback(window, xpos, ypos):
    global cameraFront, lastX, lastY, firstMouse, yaw, pitch
   
    if (firstMouse):

        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos # reversed since y-coordinates go from bottom to top
    lastX = xpos
    lastY = ypos

    sensitivity = 0.05 # change this value to your liking
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset
    pitch += yoffset

    # make sure that when pitch is out of bounds, screen doesn't get flipped
    if (pitch > 89.0):
        pitch = 89.0
    if (pitch < -89.0):
        pitch = -89.0

    front = glm.vec3()
    front.x = glm.cos(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    front.y = glm.sin(glm.radians(pitch))
    front.z = glm.sin(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

# glfw: whenever the mouse scroll wheel scrolls, this callback is called
# ----------------------------------------------------------------------
def scroll_callback(window, xoffset, yoffset):
    global fov

    fov -= yoffset
    if (fov < 1.0):
        fov = 1.0
    if (fov > 45.0):
        fov = 45.0
    
glfw.set_key_callback(window,key_event)
glfw.set_framebuffer_size_callback(window, framebuffer_size_callback)
glfw.set_cursor_pos_callback(window, mouse_callback)
glfw.set_scroll_callback(window, scroll_callback)

# tell GLFW to capture our mouse
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

In [5]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
       
    # aplicando translacao (terceira operação a ser executada)
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao (segunda operação a ser executada)
    if angle!=0:
        matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala (primeira operação a ser executada)
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp)
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(fov), largura/altura, 0.1, 100.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

In [6]:
global vertices_list
vertices_list = []  
loc_color = glGetUniformLocation(program, "color")  

In [7]:
def carregar_letra_s():
  verts = [
    (1.0, 3.0, 1.0),
    (2.0, 1.0, 1.0),
    (2.0, 2.0, 1.0),
    
    (2.0, 2.0, 1.0),
    (2.0, 1.0, 1.0),
    (3.0, 2.0, 1.0),
    
    (3.0, 2.0, 1.0),
    (2.0, 1.0, 1.0),
    (3.0, 1.0, 1.0),
    
    (3.0, 1.0, 1.0),
    (3.0, 2.0, 1.0),
    (4.0, 3.0, 1.0),
    
    (4.0, 3.0, 1.0),
    (3.0, 4.0, 1.0),
    (3.0, 5.0, 1.0),
    
    (3.0, 5.0, 1.0),
    (3.0, 4.0, 1.0),
    (2.0, 4.0, 1.0),
    
    (2.0, 4.0, 1.0),
    (3.0, 5.0, 1.0),
    (2.0, 5.0, 1.0),
    
    (2.0, 5.0, 1.0),
    (2.0, 4.0, 1.0),
    (1.0, 6.0, 1.0),
    
    (1.0, 6.0, 1.0),
    (2.0, 7.0, 1.0),
    (2.0, 8.0, 1.0),
    
    (2.0, 8.0, 1.0),
    (2.0, 7.0, 1.0),
    (3.0, 8.0, 1.0),
    
    (3.0, 8.0, 1.0),
    (2.0, 7.0, 1.0),
    (3.0, 7.0, 1.0),
    
    (3.0, 7.0, 1.0),
    (3.0, 8.0, 1.0),
    (4.0, 6.0, 1.0),
  ]
  
  vi = len(vertices_list)
  for v in verts:
    vertices_list.append(v)

  vf = len(vertices_list)

  return vi, vf - vi

In [8]:
def carregar_letra_e():
  verts = [
    (1.0, 1.0, 1.0),
    (4.0, 1.0, 1.0),
    (4.0, 2.0, 1.0),
    
    (4.0, 2.0, 1.0),
    (1.0, 2.0, 1.0),
    (1.0, 1.0, 1.0),
    
    (1.0, 1.0, 1.0),
    (2.0, 1.0, 1.0),
    (1.0, 8.0, 1.0),
    
    (1.0, 8.0, 1.0),
    (1.0, 2.0, 1.0),
    (2.0, 8.0, 1.0),
    
    (2.0, 8.0, 1.0),
    (2.0, 7.0, 1.0),
    (4.0, 8.0, 1.0),
    
    (4.0, 8.0, 1.0),
    (4.0, 7.0, 1.0),
    (2.0, 7.0, 1.0),
    
    (2.0, 7.0, 1.0),
    (1.0, 8.0, 1.0),
    (1.0, 5.0, 1.0),
    
    (1.0, 5.0, 1.0),
    (1.0, 4.0, 1.0),
    (4.0, 5.0, 1.0),
    
    (4.0, 5.0, 1.0),
    (4.0, 4.0, 1.0),
    (1.0, 4.0, 1.0),
  ]
  
  vi = len(vertices_list)
  for v in verts:
    vertices_list.append(v)

  vf = len(vertices_list)

  return vi, vf - vi

In [9]:
def carregar_letra_m():
  verts = [
    (1.0, 1.0, 1.0),
    (1.0, 8.0, 1.0),
    (2.0, 1.0, 1.0),
    
    (2.0, 1.0, 1.0),
    (1.0, 8.0, 1.0),
    (2.0, 8.0, 1.0),
    
    (2.0, 8.0, 1.0),
    (2.0, 7.0, 1.0),
    (2.5, 5.0, 1.0),
    
    (2.5, 5.0, 1.0),
    (3.0, 7.0, 1.0),
    (3.0, 8.0, 1.0),
    
    (3.0, 8.0, 1.0),
    (4.0, 8.0, 1.0),
    (3.0, 1.0, 1.0),
    
    (3.0, 1.0, 1.0),
    (4.0, 1.0, 1.0),
    (4.0, 8.0, 1.0),
  ]
  
  vi = len(vertices_list)
  for v in verts:
    vertices_list.append(v)

  vf = len(vertices_list)

  return vi, vf - vi

In [10]:
def carregar_letra_i():
  verts = [
    (1.0, 1.0, 1.0),
    (2.0, 1.0, 1.0),
    (1.0, 8.0, 1.0),
    
    (1.0, 8.0, 1.0),
    (2.0, 8.0, 1.0),
    (2.0, 1.0, 1.0),
  ]
  
  vi = len(vertices_list)
  for v in verts:
    vertices_list.append(v)

  vf = len(vertices_list)

  return vi, vf - vi

In [11]:
def carregar_letra_n():
  verts = [
    (1.0, 1.0, 1.0),
    (2.0, 1.0, 1.0),
    (1.0, 8.0, 1.0),
    
    (1.0, 8.0, 1.0),
    (2.0, 1.0, 1.0),
    (2.0, 8.0, 1.0),

    (2.0, 8.0, 1.0),
    (2.0, 6.0, 1.0),
    (3.0, 1.0, 1.0),

    (3.0, 1.0, 1.0),
    (3.0, 3.0, 1.0),
    (2.0, 8.0, 1.0),

    (3.0, 1.0, 1.0),
    (3.0, 8.0, 1.0),
    (4.0, 1.0, 1.0),

    (4.0, 1.0, 1.0),
    (4.0, 8.0, 1.0),
    (3.0, 8.0, 1.0),
  ]
  
  vi = len(vertices_list)
  for v in verts:
    vertices_list.append(v)

  vf = len(vertices_list)

  return vi, vf - vi

In [12]:
def carregar_letra_t():
  verts = [
    (2.0, 1.0, 1.0),
    (3.0, 1.0, 1.0),
    (2.0, 7.0, 1.0),
    
    (2.0, 7.0, 1.0),
    (3.0, 1.0, 1.0),
    (3.0, 7.0, 1.0),

    (3.0, 7.0, 1.0),
    (4.0, 8.0, 1.0),
    (4.0, 7.0, 1.0),

    (4.0, 7.0, 1.0),
    (4.0, 8.0, 1.0),
    (1.0, 7.0, 1.0),

    (1.0, 7.0, 1.0),
    (1.0, 8.0, 1.0),
    (4.0, 8.0, 1.0),
  ]
  
  vi = len(vertices_list)
  for v in verts:
    vertices_list.append(v)

  vf = len(vertices_list)

  return vi, vf - vi

In [13]:
def carregar_letra_r():
  verts = [
    (2.0, 8.0, 1.0),
    (1.0, 1.0, 1.0),
    (1.0, 8.0, 1.0),
    
    (1.0, 1.0, 1.0),
    (2.0, 1.0, 1.0),
    (2.0, 8.0, 1.0),

    (2.0, 8.0, 1.0),
    (3.0, 8.0, 1.0),
    (2.0, 7.0, 1.0),

    (2.0, 7.0, 1.0),
    (3.0, 7.0, 1.0),
    (3.0, 8.0, 1.0),

    (3.0, 8.0, 1.0),
    (3.0, 7.0, 1.0),
    (4.0, 6.0, 1.0),
    
    (4.0, 6.0, 1.0),
    (3.0, 5.0, 1.0),
    (3.0, 4.0, 1.0),

    (3.0, 4.0, 1.0),
    (3.0, 5.0, 1.0),
    (2.0, 4.0, 1.0),

    (2.0, 4.0, 1.0),
    (2.0, 5.0, 1.0),
    (3.0, 5.0, 1.0),

    (2.0, 4.0, 1.0),
    (3.0, 4.0, 1.0),
    (4.0, 1.0, 1.0),

    (4.0, 1.0, 1.0),
    (3.0, 1.0, 1.0),
    (2.0, 4.0, 1.0),
  ]
  
  vi = len(vertices_list)
  for v in verts:
    vertices_list.append(v)

  vf = len(vertices_list)

  return vi, vf - vi

In [14]:
def carregar_dino():
  verts = [
    # corpo
    (2.0, 4.0, 1.0),
    (1.0, 5.0, 1.0),
    (1.0, 3.0, 1.0),
    
    (2.0, 4.0, 1.0),
    (1.0, 3.0, 1.0),
    (3.0, 1.0, 1.0),

    (2.0, 4.0, 1.0),
    (3.0, 1.0, 1.0),
    (5.0, 1.0, 1.0),

    (2.0, 4.0, 1.0),
    (5.0, 1.0, 1.0),
    (6.0, 2.0, 1.0),

    (2.0, 4.0, 1.0),
    (6.0, 2.0, 1.0),
    (6.0, 6.0, 1.0),
    
    (2.0, 4.0, 1.0),
    (6.0, 6.0, 1.0),
    (4.0, 7.0, 1.0),

    # membros

    (6.0, 3.0, 1.0),
    (6.0, 4.0, 1.0),
    (7.0, 3.0, 1.0),

    (7.0, 3.0, 1.0),
    (6.0, 4.0, 1.0),
    (7.0, 4.0, 1.0),

    (3.0, 0.0, 1.0),
    (3.0, 1.0, 1.0),
    (4.0, 0.0, 1.0),

    (5.0, 0.0, 1.0),
    (5.0, 1.0, 1.0),
    (6.0, 0.0, 1.0),


    # cabeça
    (6.0, 7.0, 1.0),
    (5.0, 6.0, 1.0),
    (5.0, 10.0, 1.0),

    (5.0, 10.0, 1.0),
    (5.0, 6.0, 1.0),
    (6.0, 8.0, 1.0),

    (6.0, 8.0, 1.0),
    (6.0, 9.0, 1.0),
    (5.0, 10.0, 1.0),

    (5.0, 10.0, 1.0),
    (6.0, 9.0, 1.0),
    (7.0, 9.0, 1.0),

    (5.0, 10.0, 1.0),
    (6.0, 9.0, 1.0),
    (6.0, 10.0, 1.0),

    (7.0, 9.0, 1.0),
    (6.0, 10.0, 1.0),
    (9.0, 9.0, 1.0),

    (9.0, 9.0, 1.0),
    (6.0, 10.0, 1.0),
    (9.0, 11.0, 1.0),

    (9.0, 11.0, 1.0),
    (6.0, 10.0, 1.0),
    (8.0, 12.0, 1.0),

    (8.0, 12.0, 1.0),
    (6.0, 10.0, 1.0),
    (6.0, 11.0, 1.0),

    (6.0, 11.0, 1.0),
    (8.0, 12.0, 1.0),
    (5.0, 12.0, 1.0),

    (5.0, 12.0, 1.0),
    (6.0, 11.0, 1.0),
    (4.0, 11.0, 1.0),

    (4.0, 11.0, 1.0),
    (5.0, 11.0, 1.0),
    (4.0, 7.0, 1.0),

    (5.0, 11.0, 1.0),
    (5.0, 10.0, 1.0),
    (4.0, 7.0, 1.0),

    (4.0, 7.0, 1.0),
    (5.0, 10.0, 1.0),
    (5.0, 6.0, 1.0),

    (5.0, 6.0, 1.0),
    (6.0, 6.0, 1.0),
    (6.0, 8.0, 1.0),

    (6.0, 8.0, 1.0),
    (6.0, 6.0, 1.0),
    (7.0, 6.0, 1.0),

    (7.0, 6.0, 1.0),
    (6.0, 8.0, 1.0),
    (7.0, 8.0, 1.0),
]
  
  vi = len(vertices_list)
  for v in verts:
    vertices_list.append(v)

  vf = len(vertices_list)

  return vi, vf - vi

In [15]:
def carregar_lua():
  verts = []
  r = 5
  y_shift = 40.0
  x_shift = 20.0
  dt = 0.1
  for i in range(0,63):

    if (i >= 10 and i <= 50):
      verts.append((x_shift, y_shift, 0.0))
      verts.append((x_shift + r*np.cos(dt), y_shift + r*np.sin(dt), 0.0))
    else: dt += 0.1

    if (i >= 10 and i <= 50):
      dt += 0.1
      verts.append((x_shift + r*np.cos(dt), y_shift + r*np.sin(dt), 0.0))

  vi = len(vertices_list)
  for v in verts:
    vertices_list.append(v)

  vf = len(vertices_list)

  return vi, vf - vi

In [16]:
def desenhar(t_x, t_y, t_z, vi, vt):
    
    global vertices
    
    angle = 0.0
    
    r_x = 0.0; r_y = 0.0; r_z = 1.0
    
    s_x = 0.5; s_y = 0.5; s_z = 0.5
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    # desenha o mario
    glDrawArrays(GL_TRIANGLES, vi, vt) ## renderizando
    glUniform4f(loc_color, 0, 0, 1, 1.0) ### vermelho

In [17]:
vi_s0, vt_s0 = carregar_letra_s()
vi_e1, vt_e1 = carregar_letra_e()
vi_m2, vt_m2 = carregar_letra_m()
vi_i3, vt_i3 = carregar_letra_i()
vi_n4, vt_n4 = carregar_letra_n()
vi_t5, vt_t5 = carregar_letra_t()
vi_e6, vt_e6 = carregar_letra_e()
vi_r7, vt_r7 = carregar_letra_r()
vi_n8, vt_n8 = carregar_letra_n()
vi_e9, vt_e9 = carregar_letra_e()
vi_t10, vt_t10 = carregar_letra_t()

vi_dino, vt_dino = carregar_dino()
vi_lua, vt_lua = carregar_lua()

In [18]:
buffer_VBO = glGenBuffers(1)

In [19]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 2, GL_FLOAT, False, stride, offset)

In [20]:
glfw.show_window(window)
glEnable(GL_DEPTH_TEST) ### importante para 3D
   
while not glfw.window_should_close(window):

    currentFrame = glfw.get_time()
    deltaTime = currentFrame - lastFrame
    lastFrame = currentFrame

    glfw.poll_events() 
       
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    # glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    glPolygonMode(GL_FRONT_AND_BACK,GL_FILL) # FILL triangles

    desenhar(t_x = -11, t_y = -1.5, t_z = -20, vi=vi_s0, vt=vt_s0)
    desenhar(t_x = -9, t_y = -1.5, t_z = -20, vi=vi_e1, vt=vt_e1)
    desenhar(t_x = -7, t_y = -1.5, t_z = -20, vi=vi_m2, vt=vt_m2)
    
    desenhar(t_x = -3, t_y = -1.5, t_z = -20, vi=vi_i3, vt=vt_i3)
    desenhar(t_x = -2, t_y = -1.5, t_z = -20, vi=vi_n4, vt=vt_n4)
    desenhar(t_x = 0, t_y = -1.5, t_z = -20, vi=vi_t5, vt=vt_t5)
    desenhar(t_x = 2, t_y = -1.5, t_z = -20, vi=vi_e6, vt=vt_e6)
    desenhar(t_x = 4, t_y = -1.5, t_z = -20, vi=vi_r7, vt=vt_r7)
    desenhar(t_x = 6, t_y = -1.5, t_z = -20, vi=vi_n8, vt=vt_n8)
    desenhar(t_x = 8, t_y = -1.5, t_z = -20, vi=vi_e9, vt=vt_e9)
    desenhar(t_x = 10, t_y = -1.5, t_z = -20, vi=vi_t10, vt=vt_t10)

    desenhar(t_x = -3, t_y = -10, t_z = -20, vi=vi_dino, vt=vt_dino)
    desenhar(t_x = -3, t_y = -10, t_z = -20, vi=vi_lua, vt=vt_lua)
    
    glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)

    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    glfw.swap_buffers(window)

glfw.terminate()